In [1]:
from pfd_toolkit import Cleaner, LLM
from dotenv import load_dotenv
import os
import pandas as pd

load_dotenv("api.env")
openai_api_key = os.getenv("OPENAI_API_KEY")

# Import reports
reports = pd.read_csv('../data/testreports.csv')

llm_client = LLM(api_key=openai_api_key,
                 model="gpt-4.1-nano")

cleaner = Cleaner(
    llm=llm_client,
    reports=reports)

cleaned_reports = cleaner.clean_reports()



Processing Fields: 100%|██████████| 6/6 [00:47<00:00,  7.98s/it]


In [3]:
cleaned_reports.head()

cleaned_reports.to_csv('../data/testreports_cleaned.csv')

In [2]:
cleaner.cleaned_reports

,Unnamed: 0,URL,ID,Date,CoronerName,Area,Receiver,InvestigationAndInquest,CircumstancesOfDeath,MattersOfConcern
0,0,https://www.judiciary.uk/prevention-of-future-...,2025-0140,2025-02-01,Lydia Brown,West London,Revon Healthcare,The investigation commenced on 2023-12-18. The...,James was found deceased in his room at [REDAC...,There was a real risk that the observations co...
1,1,https://www.judiciary.uk/prevention-of-future-...,2025-0136,2025-11-03,Susan Ridge,Surrey,HMPPS,N/A: Not found,During the course of the inquest the court hea...,Probation staff are not always aware of or hav...
2,2,https://www.judiciary.uk/prevention-of-future-...,2025-0121,2025-04-03,Nicholas Walker,"Hampshire, Portsmouth and Southampton",National Institute for Health and Care Excelle...,The investigation was commenced on 2023-09-19 ...,Chloe Elizabeth Burgess was found deceased at ...,The potential dangers of the combination of am...
3,3,https://www.judiciary.uk/prevention-of-future-...,2025-0115,2025-02-28,Andrew Cox,Cornwall and the Isles of Scilly,Secretary of State for Health & Social Care,2025-02-27 I concluded a four-day jury inquest...,Mr Campbell died in hospital due to cardiac ar...,Delay in ambulance response attributable to de...
4,4,https://www.judiciary.uk/prevention-of-future-...,2025-0114,2025-02-28,Andrew Cox,Cornwall and the Isles of Scilly,Chief Constable Devon & Cornwall Constabulary;...,"On 27/2/25, I concluded a four-day jury inques...",Mr Campbell had a history of recreational drug...,Delays in ambulance attendance. Information sh...
5,5,https://www.judiciary.uk/prevention-of-future-...,2025-0113,2025-02-28,Heath Westerman,"Shropshire, Telford & Wrekin",NHS England; Chief Executive of Shrewsbury and...,"Mr Ellery, H.M. Senior Coroner for Shropshire,...",Mr Green was admitted to The Royal Shrewsbury ...,Once any patient at The Royal Shrewsbury Hospi...
6,6,https://www.judiciary.uk/prevention-of-future-...,2025-0110,2025-02-27,Richard Middleton,Dorset,Home Office,The investigation was commenced on 2024-06-13 ...,Mr Leatham-Prosser had started misusing ketami...,N/A: Not found
7,7,https://www.judiciary.uk/prevention-of-future-...,2025-0057,2025-01-31,Joseph Turner,"West Sussex, Brighton and Hove",Ministry of Defence,The investigation commenced on 2023-11-01 and ...,Mr Taylor had rapidly fallen into drug addicti...,When found to have taken illicit drugs months ...
8,8,https://www.judiciary.uk/prevention-of-future-...,2025-0055,2025-01-31,Nigel Parsley,Suffolk,Secretary of State Department of Health and So...,The investigation commenced on 2024-05-13 and ...,Kim Robinson's death was recognised at 05:16 o...,The matters of concern are as follows: the nee...
9,9,https://www.judiciary.uk/prevention-of-future-...,2025-0048,2025-01-24,Xavier Mooyaart,South London,NHS England,The investigation into the death of Charlie Ma...,Mr Marriage had a longstanding diagnosis of id...,There are cohorts of patients who are medicati...
